# Red Neuronal

## Imports

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

## Dataset

In [ ]:
# Configuramos las rutas de los directorios de las imágenes que vamos a utilizar para la red
path = "path"

# Configuramos el tamaño de las imágenes y el batch_size
img_size = (225, 225)
batch_size = 48

# Declaramos las variables donde vamos a guardar los resultados
img_names = []
img_predictions = []

# Creamos los datasets de entrenamiento y validación y los configuramos
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0 / 255.0, validation_split = 0.2)

train_generator = datagen.flow_from_directory(
    path,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = "sparse",
    color_mode = "rgb",
    subset = "training"
)

validation_generator = datagen.flow_from_directory(
    path,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = "sparse",
    color_mode = "rgb",
    subset = "validation"
)

# Guardamos los nombres de cada clase para luego asignarlos
class_indices = train_generator.class_indices

index_to_class = {v: k for k, v in class_indices.items()}

print("Set de entrenamiento: ", len(train_generator))
print("Set de validacion: ", len(validation_generator))

plt.imshow(train_generator[0][0][0])

plt.show()

## Modelo

### Estructura

In [ ]:
def create_model():
    input = tf.keras.layers.Input(shape=(img_size[0], img_size[1], 3))
    x = tf.keras.layers.Flatten()(input)

    x = tf.keras.layers.Dense(128, activation = 'relu')(x)

    x = tf.keras.layers.Dense(64, activation = 'relu')(x)

    output = tf.keras.layers.Dense(len(class_indices), activation = 'softmax')(x)

    return tf.keras.models.Model(input, output)

### Funciones

In [ ]:
def plot_metric(data, what):
  plt.figure(figsize=(10, 5))
  plt.plot(data[what])
  plt.plot(data['val_' + what])
  plt.title('model ' + what)
  plt.ylabel(what)
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')

def plot_losses(history):
    plt.rcParams['figure.figsize'] = [20, 5]
    f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)

    ax1.set_title('Losses')
    ax1.set_xlabel('epoch')
    ax1.legend(loc="upper right")
    ax1.grid()
    ax1.plot(history['loss'], label='Training loss')
    ax1.plot(history['val_loss'], label='Validation loss')
    ax1.legend()

    ax2.set_title('Accuracy')
    ax2.set_xlabel('epoch')
    ax2.legend(loc="upper right")
    ax2.grid()
    ax2.plot(history['accuracy'], label='Training accuracy')
    ax2.plot(history['val_accuracy'], label='Validation accuracy')
    ax2.legend()

    plt.show()

# Entrenamiento del Modelo

### Configuración

In [ ]:
epochs = 10

model = create_model()
model.summary()

### Compilación con Función de Pérdida

In [ ]:
model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

### Entrenamiento

In [ ]:
history = model.fit(train_generator, epochs = epochs, validation_data = validation_generator)

### Evaluación

In [ ]:
plot_losses(history.history)

# Guardar el Modelo

In [ ]:
path = "path"
model.save(path + "//" + "pesosRedNeuronal.h5")